ENCORE A FAIRE: mettre les imports tout en haut !!

La première étape de notre travail a été de récupérer les diverses données dont nous avions besoin afin d'essayer d'estimer l'impact des influenceurs de Youtube mais aussi des prix sur les livres les plus lus.

PARTIE 1: Récupérer des données sur les livres les plus lus, de différente manière

Récupération de l'API des bibliothèques de Paris afin d'estimer les cent livres les plus empruntés

Tout d'abord, formulons notre requête à l'API. On met comme limite 100 ouvrages car il s'agit de la limite de collecte possible au sein de l'API

In [96]:
import requests

response = requests.get("https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/les-1000-titres-les-plus-reserves-dans-les-bibliotheques-de-pret/records?limit=100")

# Afficher le type de contenu et le texte brut de la réponse
print("Type de contenu :", response.headers.get("Content-Type"))

Type de contenu : application/json; charset=utf-8


Une fois l'API récupérée, on observe la structure du jeu de données pour savoir comment s'appelle la variable 'résultat' ( ici 'results') à appeler afin de créer notre dataframe panda.
On observe que tous les rangs ne sont pas dans la base. Notre hypothèse optimiste est qu'il y a peut-être des types d'ouvrage non classés dans cette base ( et ne nous intéressant pas puisque nous nous intéressons seulement aux livres, dont la catégorie existe dans la base). C'est une hypothèse qui nous semble probable étant donné que nous avons par la suite créé nous-même des trous dans le classement en enlevant les DVDs de ce dernier. Cependant, nous ne pouvons pas en être sûrs.

In [97]:
wb_bibli = response.json()  # Utilisation de .json() sur l'objet réponse
print("Structure du JSON :")
print(wb_bibli)

Structure du JSON :
{'total_count': 1000, 'results': [{'rang': 12, 'type_de_document': 'Livre adulte', 'reservations': 522.0, 'titre': "Réinventer l'amour : comment le patriarcat sabote les relations hétérosexuelles", 'auteur': 'Chollet,  Mona'}, {'rang': 13, 'type_de_document': 'Bande dessinée adulte', 'reservations': 521.0, 'titre': 'Le jeune acteur', 'auteur': 'Sattouf,  Riad'}, {'rang': 14, 'type_de_document': 'Bande dessinée adulte', 'reservations': 510.0, 'titre': 'Une jeunesse au Moyen-Orient, 1992-1994', 'auteur': 'Sattouf,  Riad'}, {'rang': 37, 'type_de_document': 'DVD tous publics', 'reservations': 337.0, 'titre': 'Dune', 'auteur': None}, {'rang': 58, 'type_de_document': 'DVD tous publics', 'reservations': 266.0, 'titre': 'Onoda. 10 000 nuits dans la jungle', 'auteur': None}, {'rang': 73, 'type_de_document': 'Nouveauté', 'reservations': 241.0, 'titre': 'Chien 51 : roman', 'auteur': 'Gaudé,  Laurent'}, {'rang': 87, 'type_de_document': 'Livre adulte', 'reservations': 225.0, 'ti

Création du dataframe.
On remarque qu'il y a parfois des catégories qui ne donnent pas assez d'information sur le type d'ouvrage, à l'image de "Nouveauté", ou encore de "None". Nous pensons qu'il peut être pertinent d'étudier l'impact différé des prix et de l'exposition sur Youtube selon le genre d'ouvrage , dans la limite du possible. Ainsi, nous avons décidé d'utiliser des méthodes de scraping pour compléter la base. Avant de faire cela, nous avons décidé d'enlever de la base tous les ouvrages de type DVDs, ces derniers ne nous intéressant pas dans le cadre de notre projet.

In [98]:
import pandas as pd
df_bibli= pd.json_normalize(wb_bibli['results'])
df_bibli.head(12)

,rang,type_de_document,reservations,titre,auteur
0,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona"
1,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad"
2,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad"
3,37,DVD tous publics,337.0,Dune,None
4,58,DVD tous publics,266.0,Onoda. 10 000 nuits dans la jungle,None
5,73,Nouveauté,241.0,Chien 51 : roman,"Gaudé, Laurent"
6,87,Livre adulte,225.0,Le gosse : roman,"Olmi, Véronique"
7,109,Bande dessinée jeunesse,196.0,Attaque au clair de lune,"Oda, Eiichiro"
8,125,DVD nouveautés tous publics,177.0,West Side Story,None
9,136,None,170.0,Etés anglais,"Howard, Elizabeth Jane"


Tout d'abord, il est nécessaire de prendre connaissance de toutes les catégories pour savoir lesquelles remplacer, desquelles se débarasser...

In [99]:
print(df_bibli["type_de_document"].unique())

['Livre adulte' 'Bande dessinée adulte' 'DVD tous publics' 'Nouveauté'
 'Bande dessinée jeunesse' 'DVD nouveautés tous publics' None
 'Nouveauté jeunesse' 'Bande dessinée ado' 'Livre ado' 'Livre jeunesse']


Comme annoncé, on ne garde que les catégories qui nous intéresse ci-dessous.

In [100]:
doc = ["Livre adulte", "Bande dessinée adulte", "Nouveauté", "Bande dessinée jeunesse", "Nouveauté jeunesse",
 "Bande dessinée ado", "Livre ado", "Livre jeunesse", None]
df_bibli= df_bibli[df_bibli["type_de_document"].isin(doc)]

In [101]:
df_bibli.head(5)

,rang,type_de_document,reservations,titre,auteur
0,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona"
1,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad"
2,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad"
5,73,Nouveauté,241.0,Chien 51 : roman,"Gaudé, Laurent"
6,87,Livre adulte,225.0,Le gosse : roman,"Olmi, Véronique"


On voit qu'il y a beaucoup de livres dans notre base dont le type n'est pas renseigné.

In [102]:
nouveau = ["Nouveauté", "Nouveauté jeunesse", None]
df_bibli_nouveau =  df_bibli[df_bibli["type_de_document"].isin(nouveau)]
print(df_bibli_nouveau)

    rang    type_de_document  reservations  \
5     73           Nouveauté         241.0   
9    136                None         170.0   
16   360                None         100.0   
19   394  Nouveauté jeunesse          95.0   
20   399           Nouveauté          95.0   
22   425  Nouveauté jeunesse          91.0   
29   501                None          84.0   
34   672           Nouveauté          71.0   
39   718                None          69.0   
40   720  Nouveauté jeunesse          68.0   
45   840                None          62.0   
46   860                None          61.0   
49   910           Nouveauté          60.0   
51   945           Nouveauté          58.0   
52   949           Nouveauté          58.0   
60    61                None         261.0   
66   217           Nouveauté         132.0   
69   233           Nouveauté         127.0   
71   275           Nouveauté         114.0   
74   333                None         103.0   
83   572                None      

Pour le scrapping, nous avons choisi de le faire sur livraddict puisqu'il s'agit d'un site où tous les titres sont en français, avec un type de fonction recherche pratique pour scrapper. Sur ce site sont de plus décrits tous les livres connus avec des catégorie toujours indiquées avec un code similaire.

Pour ce faire, nous avons d'abord générer une URL de recherche sur livraddict. On scrappe la page pour récupérer l'URL du premier résultat de la recherche

In [103]:
import requests
from bs4 import BeautifulSoup

# Fonction pour rechercher un livre sur Babelio et accéder à la page du premier résultat
def search_livraddict(book_title):
    # URL de recherche sur Livraddict
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    search_url = f'https://www.livraddict.com/search.php?t={book_title}]'

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    # Trouver le premier élément correspondant au lien d'un résultat
    page = BeautifulSoup(response, "html.parser")
    first_result = page.select_one('.listing_recherche li .item_photo a')
    if first_result is not None:
        url = first_result['href']
        full_url = f"https://www.livraddict.com{url}"
        return full_url
    else:
        return None

On scrappe cette fois la page du premier résultat de la recherche précédente pour trouver les informations du livre.

In [104]:
def type_livre(info):
    if info is None:
        type_livre="Nom du livre pas trouvé sur Livraddict"
    else:
        # Normaliser les chaînes (supprimer les espaces et convertir en minuscules)
        info_0_normalized = info[0].strip().lower()
        info_1_normalized = info[1].strip().lower()

        if info_0_normalized in ["album", "artbook/beau livre", "bande-dessinée", "comics", "manga"] and info_1_normalized in ["petite enfance", "enfance"]:
            type_livre = "Bande dessinée jeunesse"

        elif info_0_normalized in ["album", "artbook/beau livre", "bande-dessinée", "comics", "manga"] and info_1_normalized in ["adolescence"]:
            type_livre = "Bande dessinée ado"

        elif info_0_normalized in ["album", "artbook/beau livre", "bande-dessinée", "comics", "manga"] and info_1_normalized in ["young adult", "adulte"]:
            type_livre = "Bande dessinée ado"

        elif info_0_normalized in ["correspondance", "documentaire", "essai", "livre pratique", "nouvelle(s)", "poésie", "roman", "théâtre"] and info_1_normalized in ["petite enfance", "enfance"]:
            type_livre = "Livre jeunesse"

        elif info_0_normalized in ["correspondance", "documentaire", "essai", "livre pratique", "nouvelle(s)", "poésie", "roman", "théâtre"] and info_1_normalized in ["adolescence"]:
            type_livre = "Livre ado"

        elif info_0_normalized in ["correspondance", "documentaire", "essai", "livre pratique", "nouvelle(s)", "poésie", "roman", "théâtre"] and info_1_normalized in ["young adult", "adulte"]:
            type_livre = "Livre adulte"

        else: type_livre= "Introuvable"
    return type_livre


In [105]:
def obtain_type(url):
    if url is not None:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers).content
        page = BeautifulSoup(response, "html.parser")
        c_format= page.find("div", class_= "book_classif book_format mr1")
        c_lectorat= page.find("div", class_="book_classif book_lectorat mr1")
        if c_format:
            a_format = c_format.find("a")
            format_livre = a_format.text
        else:
            format_livre = "Non renseigné"
        if c_lectorat:
            a_lectorat = c_lectorat.find("a")
            lectorat = a_lectorat.text
        else: 
            lectorat = "Non renseigné"
        info = [format_livre, lectorat]
        return info
    else:
        return None

Création d'une fonction qui assemble ces deux fonctions pour qu'à partir seulement d'un titre, on obtienne le type du livre.

In [106]:
def search_type_livre(titre):
    livre = titre.split()
    titre_livre = ""
    for i in range(0, len(livre) - 1):
        titre_livre = titre_livre + livre[i] + "+"
    titre_livre = titre_livre + livre[len(livre)-1]
    url = search_livraddict(titre_livre)
    infos = obtain_type(url)
    type_de_livre = type_livre(infos)
    return type_de_livre

Maintenant, on définit pour quel type de titre quelle fonction s'applique et pour quels livres on va chercher leur type.

In [107]:
def apply_search(row):
    nouveau = ["Nouveauté", "Nouveauté jeunesse", None]
    if row["type_de_document"] in nouveau:
        parties = row["titre"].split(".")
        parties_bis = row["titre"].split(":")

        #pour les séries avec nom de série, numéro tome, nom du tome
        if len(parties)==3:
            saga = parties[0].strip()
            tome = parties[1].strip()
            nom = parties[2].strip()
            if int(tome) < 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+"0"+tome)
                if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
                    type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
            elif int(tome)>= 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
        
        #pour les séries avec que nom de série et numéro de tome
        elif len(parties)==2:
            saga = parties[0].strip()
            tome = parties[1].strip()
            if int(tome) < 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+"0"+tome)
                if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
                    type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
            elif int(tome)>= 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
        
        #pour les livre où il y a écrit le genre après le titre
        elif len(parties_bis) == 2:
            type_de_livre = search_type_livre(row["titre"])
            if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
                nom = parties_bis[0].strip()
                type_de_livre = search_type_livre(nom)

        else:
            type_de_livre = search_type_livre(row["titre"])
        return type_de_livre


    else:
        return row["type_de_document"]

On applique enfin la fonction à notre dataframe. La fonction prend entre 1 et 2 minutes.

In [108]:
df_bibli['type_de_document'] = df_bibli.apply(apply_search, axis=1)

On regarde si des livres n'ont pas été trouvés. On remarque que tous les types de livre ont bien été trouvé ! Hourra

In [109]:
erreur_recherche = ["Nom du livre pas trouvé sur Livraddict"]
df_bibli_erreur =  df_bibli[df_bibli["type_de_document"].isin(erreur_recherche)]
print(df_bibli_erreur)

Empty DataFrame
Columns: [rang, type_de_document, reservations, titre, auteur]
Index: []


In [110]:
autre_erreur = ["Introuvable"]
df_bibli_int =  df_bibli[df_bibli["type_de_document"].isin(erreur_recherche)]
print(df_bibli_int)

Empty DataFrame
Columns: [rang, type_de_document, reservations, titre, auteur]
Index: []


Maintenant, on veut réordonner la base de manière à pouvoir mettre des rangs sans trous dans les numéros sur cette dernière. On commence pour ce faire par l'ordonner selon le rang, afin ensuite de numéroter les lignes dans l'ordre d'affichage.

In [111]:
df_bibli = df_bibli.sort_values(by="rang")
df_bibli = df_bibli.reset_index(drop=True)

In [112]:
nb_books = df_bibli[df_bibli['titre'].notna()].shape[0]
print(nb_books)

86


In [113]:
df_bibli['Classement bibliothèque'] = None  # Initialiser avec None
df_bibli['Top bibliothèque'] = None

# Remplir `classement` uniquement jusqu'à l'index 85
max_index = 85  # Exemple d'index limite
df_bibli.loc[df_bibli.index <= max_index, 'Classement bibliothèque'] = range(1, min(len(df_bibli), max_index + 1) + 1)
df_bibli.loc[df_bibli.index <= max_index, 'Top bibliothèque'] = 1

On vérifie pour finir le début de notre base pour s'assurer que notre variable marche bien.

In [114]:
df_bibli.head(5)

,rang,type_de_document,reservations,titre,auteur,Classement bibliothèque,Top bibliothèque
0,5,Livre adulte,755.0,La décision : roman,"Tuil, Karine",1,1
1,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona",2,1
2,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad",3,1
3,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad",4,1
4,25,Livre adulte,398.0,Dans les brumes de Capelans,"Norek, Olivier",5,1


Création d'un fichier csv pour plus tard fusionner les bases

In [115]:
df_bibli.to_csv("Les livres les plus empruntés à Paris.csv", index=False, encoding="utf-8")

Récupération des tops des livres populaires de plusieurs sites

Les 23 livres les plus populaires en 2023 sur Babelio, on ajoute une colonne indicatrice pour qu'il y ait un 1 lors du merge de toutes les tables pour les livres qui sont dans le top Babelio.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la page à scrapper
babelio_top_2023 = "https://www.babelio.com/article/2543/Les-23-livres-les-plus-populaires-de-2023"

# on fait la requête HTTP vers la page
response = requests.get(babelio_top_2023)

# On utilise l'encodage détecté par requests
response.encoding = response.apparent_encoding

if response.status_code == 200:  # Vérifie que le site autorise le scraping
    soup = BeautifulSoup(response.text, 'html.parser')

    # Liste pour stocker les données extraites
    babelio_data = []

    # On trouve toutes les sections contenant les livres
    titles_sections = soup.find_all('span', class_='titre_global')
    for section in titles_sections:
        # On extrait le titre
        title_tag = section.find('a')
        title = title_tag.text.strip() if title_tag else None

        # On extrait l'auteur
        author_text = section.text.split("de")[-1].strip() if "de" in section.text else None
        author = author_text.split("\n")[0] if author_text else None

        # Ajouter aux données si titre et auteur sont présents
        if title and author:
            babelio_data.append([title, author])

    # On convertie les données en DataFrame
    df_babelio_data = pd.DataFrame(babelio_data, columns=['Titre', 'Auteur'])

    # Ajout de la colonne indicatrice "top_babelio"
    df_babelio_data['top_babelio'] = 1
else:
    print("Erreur dans la requête")

On regarde si tout va bien avec notre dataframe.

In [68]:
print(df_babelio_data)

                                                Titre                 Auteur  \
0                                  Les Aiguilles dor       Michael McDowell   
1                                        Triste tigre            Neige Sinno   
2                                     La Petite-Fille       Bernhard Schlink   
3       La prochaine fois que tu mordras la poussière       Panayotis Pascot   
4                                       Conte de fées           Stephen King   
5                                 Un il dans la nuit         Bernard Minier   
6                                  Un abri de fortune  fortune d'Agnès Ledig   
7                              Trois vies par semaine           Michel Bussi   
8   Im not your soulmate, tome 1 : The Perfect Match              Lyla Mars   
9                                        Sur la dalle            Fred Vargas   
10            Le Bureau d'éclaircissement des destins          Gaëlle Nohant   
11  Les Sept Surs, tome 8 : Atlas, l'hi

On voit qu'il y a quelques problèmes, mais c'est rapide à corriger à la main

In [77]:
df_babelio_data.loc[df_babelio_data['Titre'] == "Un il dans la nuit",'Titre'] = 'Un oeil dans la nuit'
df_babelio_data.loc[df_babelio_data['Auteur'] == "fortune d'Agnès Ledig",'Auteur'] = 'Agnès Ledig'
df_babelio_data.loc[df_babelio_data['Titre'] == "La Femme de ménage",'Auteur'] = 'Freida McFadden'

In [78]:
print(df_babelio_data)

                                                Titre                Auteur  \
0                                  Les Aiguilles dor      Michael McDowell   
1                                        Triste tigre           Neige Sinno   
2                                     La Petite-Fille      Bernhard Schlink   
3       La prochaine fois que tu mordras la poussière      Panayotis Pascot   
4                                       Conte de fées          Stephen King   
5                                Un oeil dans la nuit        Bernard Minier   
6                                  Un abri de fortune           Agnès Ledig   
7                              Trois vies par semaine          Michel Bussi   
8   Im not your soulmate, tome 1 : The Perfect Match             Lyla Mars   
9                                        Sur la dalle           Fred Vargas   
10            Le Bureau d'éclaircissement des destins         Gaëlle Nohant   
11  Les Sept Surs, tome 8 : Atlas, l'histoire de ..

Création du fichier csv pour plus tard merge les bases.

In [127]:
df_babelio_data.to_csv("23 livres les plus lus Babelio.csv", index=False, encoding="utf-8")

Récupération des livres les plus vendus à la Fnac entre janvier et décembre 2023. Nous avons pensé que ce serait pertinent de récupérer les Best sellers afin de diminuer le biais des autres données dont nous possédons ( les personnes fréquentant les bibliothèques de Paris ou inscrivant leurs lectures sur Babelio étant un échantillon très biaisé).

In [104]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [105]:
def scrape_titles_and_authors(url, month): # cette fonction prend pour argument l'url et le mois
    response = requests.get(url)
    response.encoding = "utf-8"  # pour garantir le bon encodage
    data = [] #on crée une liste vide où on va mettre les données scrappées

    if response.status_code == 200:  # On vérifie si la requête est un succès
        soup = BeautifulSoup(response.text, 'html.parser') #on précie la manière de parcourir le code

        # On recherche toutes les balises <h2> avec la classe "title-part"
        titles = soup.find_all('h2', class_='title-part')
        for title in titles:
            raw_text = title.text.strip()  # Texte brut pour manipulation

            # Debugging : Afficher le texte brut
            print(f"Texte brut extrait : {raw_text}")

            # On fait un regex pour extraire les informations
            match = re.match(r'^(\d+)\s[–—]\s(.+?)\s[–—]\s(.+?)(?:[,/]\s(.+?))?\s\((.+?)\)$', raw_text)
            if match:
                classement = match.group(1).strip()  # Classement
                titre = match.group(2).strip()  # Titre
                auteur_principal = match.group(3).strip()  # Auteur principal
                co_auteur_raw = match.group(4).strip() if match.group(4) else None  # Co-auteur brut (facultatif)
                maison_edition = match.group(5).strip()  # Maison d'édition

                # Traitement des co-auteurs pour les séparer en liste
                co_auteurs = []
                if co_auteur_raw:
                    co_auteurs = re.split(r',\s|/\s', co_auteur_raw)  # Diviser par virgule ou slash

                # Ajouter les informations dans la liste (sans maison d'édition)
                data.append([month, classement, titre, auteur_principal] + co_auteurs[:2])  # Max 2 co-auteurs
    else:
        print(f"Erreur lors du scraping de {month}: Status code {response.status_code}")

    return data

Liste des URL à scrapper pour récupérer tous les bestsellers de 2023.

In [106]:
urls_months = [
    ("https://leclaireur.fnac.com/selection/cp50350-top-10-les-best-sellers-du-mois-de-janvier-2023/", "Janvier"),
    ("https://leclaireur.fnac.com/selection/cp50709-top-10-les-best-sellers-du-mois-de-fevrier-2023/", "Février"),
    ("https://leclaireur.fnac.com/selection/cp43551-top-10-les-best-sellers-de-mars-2023/", "Mars"),
    ("https://leclaireur.fnac.com/selection/cp44047-top-10-les-best-sellers-davril-2023/", "Avril"),
    ("https://leclaireur.fnac.com/selection/cp48182-top-10-les-best-sellers-de-mai-2023/", "Mai"),
    ("https://leclaireur.fnac.com/selection/cp48395-top-10-les-best-sellers-de-juin-2023/", "Juin"),
    ("https://leclaireur.fnac.com/selection/cp40500-top-10-les-best-sellers-de-juillet-2023/", "Juillet"),
    ("https://leclaireur.fnac.com/selection/cp48936-top-10-les-best-sellers-du-mois-daout-2023/", "Août"),
    ("https://leclaireur.fnac.com/selection/cp49107-top-10-les-best-sellers-du-mois-de-septembre-2023/", "Septembre"),
    ("https://leclaireur.fnac.com/selection/cp49482-top-10-les-best-sellers-du-mois-doctobre-2023/", "Octobre"),
    ("https://leclaireur.fnac.com/selection/cp53758-top-10-les-best-sellers-du-mois-de-novembre-2023/", "Novembre"),
    ("https://leclaireur.fnac.com/selection/cp49831-top-10-les-best-sellers-du-mois-de-decembre-2023/", "Décembre"),
]

On construit notre base de données: changer le nom en df_bestsellers !!!

In [107]:
# On initialise la liste pour toutes les données
top_data_2023 = []

# On scrape chaque URL
for url, month in urls_months:
    print(f"Scraping pour le mois de {month}...")
    month_data = scrape_titles_and_authors(url, month)
    top_data_2023.extend(month_data)

# Conversion des données en DataFrame pandas
df_top_books_2023 = pd.DataFrame(top_data_2023, columns=["Mois", "Classement", "Titre", "Auteur", "Co_auteur1", "Co_auteur2"])

# Création de la variable indicatrice "top_fnac_1" (tous les livres ont un classement)
df_top_books_2023['top_fnac_1'] = 1

# Comptage des occurrences des livres pour créer "top_fnac_2_plus"
counts = df_top_books_2023['Titre'].value_counts()
df_top_books_2023['top_fnac_2_plus'] = df_top_books_2023['Titre'].apply(lambda x: 1 if counts[x] > 1 else 0)

# Suppression des colonnes "Mois" et "Classement", et suppression des doublons
df_top_books_2023 = df_top_books_2023.drop(columns=["Mois", "Classement"]).drop_duplicates(subset=['Titre'])

Scraping pour le mois de Janvier...
Texte brut extrait : 1 – Captive, Tome 2 – Sarah Rivens (BMR)
Texte brut extrait : 2 – Le Suppléant – Prince Harry (Fayard)
Texte brut extrait : 3 – À tout jamais – Colleen Hoover (Hugo Roman)
Texte brut extrait : 4 – Le Silence et la colère – Pierre Lemaitre (Calmann Lévy)
Texte brut extrait : 5 – Le Monde sans fin, miracle énergétique et dérive climatique – Christophe Blain, Jean-Marc Jancovici (Dargaud)
Texte brut extrait : 6 – Captive, Tome 1 – Sarah Rivens (BMR)
Texte brut extrait : 7 – Plus jamais sans moi – Maud Ankaoua (Eyrolles)
Texte brut extrait : 8 – Captive, Tome 1.5 : Perfectly Wrong – Sarah Rivens (BMR)
Texte brut extrait : 9 – Le Mage du Kremlin – Giuliano Da Empoli (Gallimard)
Texte brut extrait : 10 – Kaamelott, Tome 10, Karadoc Et L’Icosaèdre – Steven Dupré, Alexandre Astier (Casterman)
Texte brut extrait : Partagez vos coups de cœur sur le Forum des Lecteurs
Scraping pour le mois de Février...
Texte brut extrait : 1 – À tout jamai

On affiche le dataframe pour vérifier qu'il n'y a pas de problèmes

In [108]:
print(df_top_books_2023)

                                                 Titre            Auteur  \
0                                      Captive, Tome 2      Sarah Rivens   
1                                         Le Suppléant      Prince Harry   
2                                        À tout jamais    Colleen Hoover   
3                              Le Silence et la colère   Pierre Lemaitre   
4    Le Monde sans fin, miracle énergétique et déri...  Christophe Blain   
..                                                 ...               ...   
105             Largo Winch, Tome 24 : Le Centile d’or        Giacometti   
107                               Lou, Tome 2 : Sonata       Julien Neel   
108                                Ma vie sans gravité    Thomas Pesquet   
114  Mortelle Adèle et les reliques du chat lune, T...            Mr Tan   
117                               Lou ! Sonata, Tome 2       Julien Neel   

              Co_auteur1 Co_auteur2  top_fnac_1  top_fnac_2_plus  
0                   

Comme il y a trop de lignes pour afficher le dataframe, on vérifie nos données en faisant un dossier csv

In [100]:
df_top_books_2023.to_csv("best_sellers_fnac_2023_cleaned.csv", index=False, encoding="utf-8")

On corrige les quelques incohérences à la main

In [109]:
df_top_books_2023.loc[df_top_books_2023['Auteur'] == "Astérix,Hors collection : L’Empire du milieu – Fabrice Tarrin",['Titre', 'Auteur']] = ['Astérix, Hors collection : L’Empire du milieu', "Fabrice Tarrin"]
df_top_books_2023.loc[df_top_books_2023['Auteur'] == "Blake et Mortimer,Avant Blake et Mortimer T2 : La Flèche ardente – Van Hamme",['Titre', 'Auteur']] = ['Blake et Mortimer,Avant Blake et Mortimer T2 : La Flèche ardente', "– Van Hamme"]
df_top_books_2023.loc[df_top_books_2023['Titre'] == "Bleak",['Titre', 'Auteur', 'Co_auteur1']] = ['Bleak,3 histoires d’horreur,Volume 2', "Squeezie", None]

Maintenant, pour harmoniser cette base et rendre plus simple le merge, nous avons voulu mettre tous les auteurs dans la colonne Auteur et enlever les colonnes de co-auteurs

In [114]:
def column_author(row):
    if row['Co_auteur1'] is not None and row['Co_auteur2'] is not None:
        author= row['Auteur'] + ', '+ row['Co_auteur1'] + ', '+ row['Co_auteur2']
    elif row['Co_auteur1'] is not None and row['Co_auteur2'] is None:
        author= row['Auteur'] + ', '+ row['Co_auteur1']
    else:
        author = row['Auteur']
    return author

In [115]:
df_top_books_2023['Auteur'] = df_top_books_2023.apply(column_author, axis=1)

On vérifie que cela a bien marché

In [116]:
df_top_books_2023.head(10)

,Titre,Auteur,Co_auteur1,Co_auteur2,top_fnac_1,top_fnac_2_plus
0,"Captive, Tome 2",Sarah Rivens,None,None,1,1
1,Le Suppléant,Prince Harry,None,None,1,1
2,À tout jamais,Colleen Hoover,None,None,1,1
3,Le Silence et la colère,Pierre Lemaitre,None,None,1,1
4,"Le Monde sans fin, miracle énergétique et déri...","Christophe Blain, Jean-Marc Jancovici",Jean-Marc Jancovici,None,1,1
5,"Captive, Tome 1",Sarah Rivens,None,None,1,1
6,Plus jamais sans moi,Maud Ankaoua,None,None,1,1
7,"Captive, Tome 1.5 : Perfectly Wrong",Sarah Rivens,None,None,1,0
8,Le Mage du Kremlin,Giuliano Da Empoli,None,None,1,1
9,"Kaamelott, Tome 10, Karadoc Et L’Icosaèdre","Steven Dupré, Alexandre Astier",Alexandre Astier,None,1,0


On peut donc maintenant enlever les colonnes indésirables.

In [117]:
df_top_books_2023 = df_top_books_2023.drop(columns=['Co_auteur1', 'Co_auteur2'])

In [120]:
df_top_books_2023.head(5)

,Titre,Auteur,top_fnac_1,top_fnac_2_plus
0,"Captive, Tome 2",Sarah Rivens,1,1
1,Le Suppléant,Prince Harry,1,1
2,À tout jamais,Colleen Hoover,1,1
3,Le Silence et la colère,Pierre Lemaitre,1,1
4,"Le Monde sans fin, miracle énergétique et déri...","Christophe Blain, Jean-Marc Jancovici",1,1


On modifie le fichier CSV pour merge plus tard les bases.

In [128]:
df_top_books_2023.to_csv("best_sellers_fnac_2023_cleaned.csv", index=False, encoding="utf-8")

Partie 2: Récupérer des données sur des systèmes permettant de juger de la qualité des livres.
Le but est d'essayer de comprendre s'il y a une corrélation entre les livres lus et les livres jugés de qualité, mais aussi entre les livres jugés de qualité et les livres présentés dans des vidéos Youtube.

Tout d'abord, on s'intéresse aux livres sortis en 2023 les mieux notés sur la plateforme Livraddict. On en importe les titres et les auteurs.

In [ ]:
# URL de la page à scrapper
livraddict_top_2023 = "https://www.livraddict.com/prix-livraddict/2024/"

# Requête HTTP vers la page
response = requests.get(livraddict_top_2023)
response.encoding = 'utf-8'

# On vérfie que le site autorise le scraping et on scrap si oui
if response.status_code == 200:  
    soup = BeautifulSoup(response.text, 'html.parser')

    # Liste pour stocker les données extraites
    livraddict_data = []

    # On parcourt toutes les catégories
    categories = soup.find_all('div', class_='categorie_prix portlet light')
    for category in categories:
        # On extrait le nom de la catégorie et supprime "catégorie" devant si présent
        category_name = category.find('h2').text.strip()
        if "catégorie" in category_name.lower():
            category_name = category_name.lower().replace("catégorie", "").strip().capitalize()

        # On parcourt les livres de la catégorie
        books = category.find_all('div', style=lambda x: x and "margin-bottom:10px" in x)
        for book in books:
            # On extrait le titre du livre
            title_tag = book.find('h3').find('a')
            title = title_tag.find('strong').text.strip() if title_tag else None

            # On extrait l'auteur du livre
            author = title_tag.contents[-1].strip() if title_tag else None

            # On ajoute les informations à la liste créée
            if title and author:
                livraddict_data.append([category_name, title, author])

    # On convertit les données en DataFrame
    df_livraddict_data = pd.DataFrame(livraddict_data, columns=['Catégorie', 'Titre', 'Auteur'])

    # On ajoute une colonne indicatrice "top_livraddict"
    df_livraddict_data['top_livraddict'] = 1
else:
    print("Erreur {response.status_code} lors de la requête.")

On regarde la base pour s'assurer qu'il n'y ait pas d'incohérence et pour se familiariser avec

In [123]:
df_livraddict_data.head(10)

,Catégorie,Titre,Auteur,top_livraddict
0,Young adult,"Engélion, tome 1 : Illuminer les Cieux",Justine Tiphagne,1
1,Young adult,Tant que fleuriront les citronniers (As Long a...,Zoulfa Katouh,1
2,Young adult,L'effet Boule de neige,Clara Héraut,1
3,Young adult,"Écarlate sous la cendre, tome 1",Élisabeth Koshava,1
4,Young adult,Le sirénien,Capucine Sergent,1
5,Jeunesse,Wonka,Sibéal Pounder,1
6,Jeunesse,L'étoile du soir,Siècle Vaëlban,1
7,Jeunesse,"Les clans du ciel, tome 1 : La quête d'Ellie (...",Jessica Khoury,1
8,Jeunesse,"Crook Haven, tome 1 : L'école des voleurs (Cr...",J. J. Arcanjo,1
9,Jeunesse,"Sentinelles du Royaume Sauvage, tome 1",Alexandra Ott,1


Création du fichier csv

In [125]:
df_livraddict_data.to_csv('livraddict_prix_2024.csv', index=False, encoding='utf-8')

Récupération de tous les prix littéraires français connus de Wikipédia entre 2019 et 2023

Création d'un dataframe vide pour ensuite entrer les donner scrappées.

In [133]:
colonnes = ['Année', 'Prix', 'Auteur', 'Titre']

# Créer un DataFrame vide avec ces colonnes
df_wikipedia = pd.DataFrame(columns=colonnes)

print(df_wikipedia)

Empty DataFrame
Columns: [Année, Prix, Auteur, Titre]
Index: []


Scrapping sur wikipedia

In [134]:
def scrape_wikipedia_page_france(year):
    global df_wikipedia
    url = f"https://fr.wikipedia.org/wiki/Prix_litt%C3%A9raires_{year}"
    response = requests.get(url)
    response.encoding = "utf-8"  # On garantit le bon encodage

    if response.status_code == 200:  # On vérifie si la requête est un succès
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # On recherche la section France
        france_section = soup.find('h2', string="France")
        if france_section:
            # La liste <ul> qui suit contient les prix pour la France
            france_list = france_section.find_next('ul')

            if france_list:
                # On recherche tous les <li> dans la liste
                prizes = france_list.find_all('li')

                # On extrait les données
                for prize in prizes:
                    # Nom du prix
                    prize_name = prize.find('a')
                    prize_name = prize_name.text.strip() if prize_name else None

                    # Auteur
                    author = prize.find_all('a')
                    author = author[1].text.strip() if len(author) > 1 else None

                    # Titre
                    title = prize.find('i')
                    title = title.text.strip() if title else None

                    # On ajoute les résultats si les données sont complètes
                    if prize_name and author and title:
                        new_row = {'Année': year, 'Prix': prize_name, 'Auteur': author, 'Titre': title}
                        df_wikipedia = pd.concat([df_wikipedia, pd.DataFrame([new_row])], ignore_index=True)
        else:
            print(f"Section 'France' non trouvée pour l'année {year}")
    else:
        print(f"Erreur lors du scraping de l'année {year}: Status code {response.status_code}")

    return df_wikipedia

In [135]:
for year in range(2019, 2024):
    scrape_wikipedia_page_france(year)

Quand on regarde les données, on voit bien qu'il y a de gros problèmes. En regardant le fichier csv, c'est d'autant plus flagrant. Nous avons identifiés plusieurs types de problèmes, que nous avons étudié puis corrigé soit manuellement quand il ne concernait que quelques lignes, soit automatiquement, en scrappant sur un autre site

In [136]:
df_wikipedia.head(5)

,Année,Prix,Auteur,Titre
0,2019,Prix Femina,Par les routes,Par les routes
1,2019,Prix Femina étranger,Ordesa,Ordesa
2,2019,Prix Femina essai,Emmanuelle Lambert,Giono furioso
3,2019,Prix Femina des lycéens,La Chaleur,La Chaleur
4,2019,Prix Goncourt,Tous les hommes n'habitent pas le monde de la ...,Tous les hommes n'habitent pas le monde de la ...


In [137]:
df_wikipedia.to_csv("prix_litteraires.csv", index=False)

Premier type d'incohérence ( aussi le plus important): il y a eu une erreur dans le scrapping de nombreux ouvrages sur Wikipédia qui a fait qu'on a deux fois le Titre des livres au lieu d'en avoir l'auteur ou les auteurs. Ce problème concerne 81 ligne, ce qui est un nombre considérable

In [144]:
df_double = df_wikipedia[df_wikipedia['Auteur']==df_wikipedia['Titre']]
print(df_double)

    Année                                         Prix  \
0    2019                                  Prix Femina   
1    2019                         Prix Femina étranger   
3    2019                      Prix Femina des lycéens   
4    2019                                Prix Goncourt   
5    2019               Prix Goncourt du premier roman   
..    ...                                          ...   
189  2022  Grand prix du roman de l'Académie française   
218  2022                          Grand prix RTL-Lire   
223  2022                    Grand Prix Roman de l'été   
290  2023                    Grand Prix Roman de l'été   
292  2023                                   Ada Palmer   

                                                Auteur  \
0                                       Par les routes   
1                                               Ordesa   
3                                           La Chaleur   
4    Tous les hommes n'habitent pas le monde de la ...   
5            

Le deuxième grand problème que nous avons rencontré est le remplacement des noms d'auteurs par des chiffres entre crochets pour 13 lignes.

In [145]:
df_ch = df_wikipedia[df_wikipedia['Auteur'].str.contains(r'^\[\d+\]$')]
print(df_ch)

    Année                                  Prix Auteur  \
104  2020              Prix littéraire du Monde    [4]   
150  2021                   Prix Première Plume    [3]   
157  2021  Prix Eugène-Dabit du roman populiste    [5]   
180  2022        Prix Goncourt de la biographie   [10]   
209  2022                           Anne Berest   [33]   
210  2022      Prix littéraire ENS Paris-Saclay   [34]   
214  2022             Prix du Quai des Orfèvres   [38]   
217  2022  Prix Eugène-Dabit du roman populiste   [41]   
222  2022      Grand prix des lectrices de Elle   [45]   
225  2022       Grand Prix de Poésie de la SGDL   [46]   
227  2022                           Prix Wepler   [47]   
247  2023        Prix Goncourt de la biographie   [45]   
297  2023   Prix littéraire de la Ville de Caen   [70]   

                              Titre  
104                           Monde  
150  Avant que le monde ne se ferme  
157              Des gens comme eux  
180           Léopold Sédar Senghor

Il y a parfois eu des incohérences dans les lignes correspondant à des prix en particulier.

cas 1: le prix François Mauriac. Le nom complet du prix est "Prix François Mauriac de l'Académie française". La deuxième partie de son nom a remplacé tous les noms d'auteurs sensés être récoltés lors du scrapping.

In [146]:
df_mauriac = df_wikipedia[df_wikipedia['Prix']=='Prix François-Mauriac']
print(df_mauriac)

    Année                   Prix              Auteur  \
53   2019  Prix François-Mauriac  Académie française   
90   2020  Prix François-Mauriac  Académie française   
199  2022  Prix François-Mauriac  Académie française   
267  2023  Prix François-Mauriac  Académie française   

                                  Titre  
53   Là-bas, août est un mois d'automne  
90               Un automne de Flaubert  
199                                Zita  
267   À l’abri des hommes et des choses  


Cas 2: Le prix Telerama. Tous les noms de livre ont été remplacés par la chaîne de caractère "Télérama". Les noms d'auteurs ne sont pour autant pas tout le temps juste puisqu'il y a parfois dans la colonne "Auteur" le titre apparent du livre au lieu d'y avoir le nom de la personne l'ayant écrit. Du fait du manque d'uniformité de ces incohérences, nous avons décidé de les corriger manuellement. 

In [147]:
df_telerama = df_wikipedia[df_wikipedia['Prix']=='Prix France Culture-Télérama']
print(df_telerama)

    Année                          Prix           Auteur     Titre
28   2019  Prix France Culture-Télérama        La Maison  Télérama
97   2020  Prix France Culture-Télérama         Chavirer  Télérama
208  2022  Prix France Culture-Télérama   Kaouther Adimi  Télérama
276  2023  Prix France Culture-Télérama  Salma El Moumni  Télérama


Correction manuelle: nous avons aussi rajouté une ligne pusiqu'il manquait l'année 2021.

In [148]:
df_wikipedia.loc[(df_wikipedia["Prix"] == "Prix France Culture-Télérama") & (df_wikipedia["Année"] == 2019), ["Auteur", "Titre"]] = ["Emma Becker", "La Maison"]
df_wikipedia.loc[(df_wikipedia["Prix"] == "Prix France Culture-Télérama") & (df_wikipedia["Année"] == 2020), ["Auteur", "Titre"]] = ["Lola Lafon", "Chavirer"]
df_wikipedia.loc[(df_wikipedia["Prix"] == "Prix France Culture-Télérama") & (df_wikipedia["Année"] == 2022), "Titre"] = "Au vent mauvais"
df_wikipedia.loc[(df_wikipedia["Prix"] == "Prix France Culture-Télérama") & (df_wikipedia["Année"] == 2023), "Titre"] = "Adieu Tanger"
nouvelle_ligne = pd.DataFrame({'Année': [2021], 'Prix': ["Prix France Culture-Télérama"], 'Auteur':['Mathieu Palain'], 'Titre':["Ne t'arrête pas de courir "]})
df_wikipedia = pd.concat([df_wikipedia, nouvelle_ligne], ignore_index=True)

cas 3: le prix Maya. Ici, ce sont les noms d'auteur qui ont été remplacés par "animaliste", un peu comme pour le prix François Mauriac. Il est donc possible d'automatiser le traitement de cette incohérence.

In [149]:
df_maya = df_wikipedia[df_wikipedia['Prix']=='Prix Maya']
print(df_maya)

    Année       Prix      Auteur                       Titre
61   2019  Prix Maya  animaliste  Les Paupières des poissons
124  2020  Prix Maya  animaliste            Insolente Veggie


cas 4: le prix Femme Actuelle. Ici, il n'y a ni titre ni auteur: les deux colonnes ont été remplacé par "Femme actuelle". De ce fait, il sera ici aussi nécessaire de corriger manuellement les erreur.

In [150]:
df_femme_a = df_wikipedia[df_wikipedia['Titre']=='Femme Actuelle']
print(df_femme_a)

    Année                       Prix          Auteur           Titre
46   2019  Grand Prix Roman de l'été  Femme Actuelle  Femme Actuelle
109  2020  Grand Prix Roman de l'été  Femme Actuelle  Femme Actuelle
162  2021  Grand Prix Roman de l'été  Femme Actuelle  Femme Actuelle
223  2022  Grand Prix Roman de l'été  Femme Actuelle  Femme Actuelle
290  2023  Grand Prix Roman de l'été  Femme Actuelle  Femme Actuelle


Correction manuelle

In [151]:
df_wikipedia.loc[df_wikipedia["Titre"] == "Femme Actuelle", "Prix"] = "Grand prix Roman de l'été de Femme Actuelle"

In [157]:
df_wikipedia.loc[(df_wikipedia['Titre'] == 'Femme Actuelle') & (df_wikipedia['Année'] == 2019), ['Titre', 'Auteur']] = ['Au bout de la nuit', 'Bruno Bouzounie']
df_wikipedia.loc[(df_wikipedia['Titre'] == 'Femme Actuelle') & (df_wikipedia['Année'] == 2020), ['Titre', 'Auteur']] = ['La Conjonction dorée', 'Benoît Sagaro']
df_wikipedia.loc[(df_wikipedia['Titre'] == 'Femme Actuelle') & (df_wikipedia['Année'] == 2021), ['Titre', 'Auteur']] = ['Pour exister encore', 'Martine Duchesne']
df_wikipedia.loc[(df_wikipedia['Titre'] == 'Femme Actuelle') & (df_wikipedia['Année'] == 2022), ['Titre', 'Auteur']] = ["Le journal de Betty Swan", 'Cindy Valt']
df_wikipedia.loc[(df_wikipedia['Titre'] == 'Femme Actuelle') & (df_wikipedia['Année'] == 2023), ['Titre', 'Auteur']] = ['Hauteclaire', 'Dominique Delplace']

cas 5: Il y a aussi un problème avec les prix du journal le Monde, que nous avons corrigé manuellement.

In [182]:
df_monde = df_wikipedia[df_wikipedia['Titre']=='Monde']
print(df_monde)

    Année                      Prix               Auteur  Titre  prix_19_23
42   2019  Prix littéraire du Monde  Une bête au paradis  Monde           1
221  2022  Prix littéraire du Monde       Mathieu Belezi  Monde           1
288  2023  Prix littéraire du Monde          Neige Sinno  Monde           1


In [188]:
df_wikipedia.loc[(df_wikipedia['Auteur'] == "Une bête au paradis") & (df_wikipedia['Titre'] == "Monde"),['Auteur','Titre']] = ['Cécile Coulon','Une bête au paradis']
df_wikipedia.loc[(df_wikipedia['Auteur'] == "Mathieu Belezi") & (df_wikipedia['Titre'] == "Monde"),'Titre'] = ['Attaquer la terre et le soleil']
df_wikipedia.loc[(df_wikipedia['Auteur'] == "Neige Sinno") & (df_wikipedia['Titre'] == "Monde"),'Titre'] = ['Triste tigre']

Enfin, nous avons fait la remarque plus de l'ordre de la mise en page qu'il y a parfois dans les cases des précisions entre parenthèses ou crochets sur l'auteur ou le titre, précisions qui ne font pas partie du nom ni du titre de l'ouvrage. Ainsi, pour faciliter la future opération merge, nous avons décidé d'enlever ces choses de trop.

In [158]:
df_wikipedia['Auteur'] = df_wikipedia['Auteur'].str.replace(r'\(.*\)', '', regex=True) #on enlève les parenthèses après les noms d'auteur
df_wikipedia['Auteur'] = df_wikipedia['Auteur'].str.replace(r'\[.*\]', '', regex=True) # on enlève aussi les éventuels crochets
df_wikipedia['Titre'] = df_wikipedia['Titre'].str.replace(r'\[.*\]', '', regex=True)

Il y a aussi des prix dont le nom nous a paru bizarre, mais étant donné que nous n'utilisons pas les noms des prix et construisons seulement une indicatrice indiquant si le livre a reçu un prix ou pas, nous n'en avons pas tenu compte

Nous avons commencé par compléter la base en scrappant Gallimard qui est un site fiable et bien codé. Le problème étant qu'il n'y a que les livres édités chez Gallimard sur ce site et donc qu'il faudra scrapper autre part pour compléter la base.

In [154]:
import time
import re
import requests
from bs4 import BeautifulSoup as soup

def search_gallimard(book_title, year):
    #reformulation du titre en différente partie
    parts = re.findall(r'\w+|[^\w\s]', book_title)
    book_title_gen = book_title.lower().strip()

    # URL de recherche sur Livraddict
    search_title = '+'.join(parts)
    lower_search_title = '%20'.join(parts).lower()
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:118.0) Gecko/20100101 Firefox/118.0'}
    search_url = f'https://www.gallimard.fr/catalogue?search_api_fulltext={search_title}&field_date_de_parution_1%5Bdate%5D={year}-01-01&field_date_de_parution_2%5Bdate%5D={year}-12-31&field_prix%5Bmin%5D=&field_prix%5Bmax%5D=&title_3=&items_per_page=20&sort_bef_combine=field_identifiant_catalogue_ASC'
    time.sleep(5)

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    response_text = response.decode('utf-8')
    # Trouver le premier élément correspondant au lien d'un résultat
    page = soup(response_text, "html.parser")
    results = page.find_all("div", class_ = "card--ouvrage")
    author = ''
    for i in range(len(results)):
        result = results[i]
        div_title = result.find("div", class_ = "title")
        title = div_title.text
        title_gen = title.lower().strip()
        if book_title_gen == title_gen:
            div_author = result.find('div', class_='paragraph paragraph--type--auteur paragraph--view-mode--links')
            a_author = div_author.find("a")
            author = a_author.text
    if author == '':
        author = "Pas trouvé sur Gallimard"
    return author

In [155]:
def search_author(row):
    if row["Auteur"] == row["Titre"] or re.match(r'^\[\d+\]$', row['Auteur']) or row['Prix']=='Prix François-Mauriac' or row['Prix']=='Prix Maya':
        author = search_gallimard(row["Titre"], row["Année"])
        if author == None:
            author = "Problème dans la fonction?"
        return author
    else:
        return row["Auteur"]

L'opération prend entre 7 et 8 minutes.

In [159]:
df_wikipedia['Auteur'] = df_wikipedia.apply(search_author, axis=1)

Comme nous l'avons indiqué plus haut, pour compléter ces données, nous avons scrapper sur le site de la Maison du Livre.

In [160]:
def search_maison_du_livre(book_title, year):
    #reformulation du titre en différente partie
    parts = re.findall(r'\w+|[^\w\s]', book_title)
    book_title_gen = book_title.lower().strip()

    # URL de recherche sur Livraddict
    search_title = '+'.join(parts)
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:118.0) Gecko/20100101 Firefox/118.0'}
    search_url = f'https://www.maisondulivre.com/listeliv.php?base=paper&mots_recherche={search_title}+{year}'
    time.sleep(5)

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    response_text = response.decode('utf-8')

    # Trouver le premier élément correspondant au lien d'un résultat
    page = soup(response_text, "html.parser")
    liste_livres = page.find('ul', {'id': 'liste_livres'})
    livres = liste_livres.find_all("li")
    author = ''
    for i in range(len(livres)):
        livre = livres[i]
        h2_titre = livre.find("h2", class_="livre_titre")
        if h2_titre:
            a_titre = h2_titre.find("a")
            titre = a_titre.get_text(strip=True)
            title_adapted = re.sub(r'\s?\(.*?\)', '', titre)
            title_gen = title_adapted.lower().strip()
            if title_gen == book_title_gen:
                h2_auteur = livre.find("h2", class_ = "livre_auteur")
                if h2_auteur == None:
                    auteur = "Auteur non renseigné"
                else:
                    a_auteur = h2_auteur.find("a")
                    author = a_auteur.get_text(strip=True)
    if author == '':
        author = "Livre pas trouvé"
    return author

In [161]:
def complete_author(row):
    if row["Auteur"] == "Pas trouvé sur Gallimard":
        author = search_maison_du_livre(row["Titre"], row["Année"])
        if author == None:
            author = "Problème dans la fonction?"
        return author
    else:
        return row["Auteur"]

L'opération ci-dessous prend à peu près 7 minutes.

In [162]:
df_wikipedia['Auteur'] = df_wikipedia.apply(complete_author, axis=1)

Regardons la base de données pour vérifier qu'il n'y a plus d'incohérence et éventuellement les corriger

In [163]:
df_wikipedia.head(10)

,Année,Prix,Auteur,Titre
0,2019,Prix Femina,Sylvain Prudhomme,Par les routes
1,2019,Prix Femina étranger,Manuel Vilas,Ordesa
2,2019,Prix Femina essai,Emmanuelle Lambert,Giono furioso
3,2019,Prix Femina des lycéens,Victor Jestin,La Chaleur
4,2019,Prix Goncourt,Jean-Paul Dubois,Tous les hommes n'habitent pas le monde de la ...
5,2019,Prix Goncourt du premier roman,Marie Gauthier,Court vêtue
6,2019,Prix Goncourt des lycéens,Karine Tuil,Les Choses humaines
7,2019,Prix Goncourt de la nouvelle,Caroline Lamarche,Nous sommes à la lisière
8,2019,Prix Goncourt de la biographie,Emily Dickinson,"Manifeste incertain, volume 7 : Emily Dickinso..."
9,2019,Choix Goncourt de la Pologne,Louis-philippe Dalembert,Mur Méditerranée


Comme on le voit ci-dessous, il reste encore quelques livres qui n'ont pas été trouvés. Comme il n'y en a pas beaucoup, on corrige manuellement la base.

In [165]:
erreur_recherche = ["Livre pas trouvé", ""]
df_verif =  df_wikipedia[df_wikipedia["Auteur"].isin(erreur_recherche)]
print(df_verif)

    Année                                  Prix            Auteur  \
14   2019                    Prix Médicis essai  Livre pas trouvé   
17   2019       Prix Renaudot du livre de poche  Livre pas trouvé   
24   2019                         Prix Décembre  Livre pas trouvé   
48   2019                        Jacques Collin  Livre pas trouvé   
61   2019                             Prix Maya  Livre pas trouvé   
75   2020                          Prix Médicis  Livre pas trouvé   
78   2020                         Prix Renaudot  Livre pas trouvé   
88   2020                 Prix du premier roman  Livre pas trouvé   
95   2020                         Prix de Flore  Livre pas trouvé   
104  2020              Prix littéraire du Monde                     
107  2020                           Tess Sharpe                     
124  2020                             Prix Maya  Livre pas trouvé   
150  2021                   Prix Première Plume                     
152  2021                         

Corrections manuelles

In [176]:
df_wikipedia.loc[df_wikipedia['Titre'] == "J'ai oublié", 'Auteur'] = 'Bulle Ogier'
df_wikipedia.loc[df_wikipedia['Titre'] == "Une vieille histoire. Nouvelle version", 'Auteur'] = 'Jonathan Littell'
df_wikipedia.loc[df_wikipedia['Titre'] == "Les Grands Cerfs", 'Auteur'] = 'Claudie Hunzinger'
df_wikipedia.loc[df_wikipedia['Titre'] == "Anatèm", 'Auteur'] = 'Neal Stephenson'
df_wikipedia.loc[df_wikipedia['Titre'] == "Le Cœur synthétique", 'Auteur'] = 'Chloé Delaume'
df_wikipedia.loc[df_wikipedia['Titre'] == "Terra Ignota", 'Auteur'] = 'Ada Palmer'
df_wikipedia.loc[df_wikipedia['Titre'] == "Histoire du fils", 'Auteur'] = 'Marie-Hélène Lafon'
df_wikipedia.loc[df_wikipedia['Titre'] == "Adios Cow Boy", 'Auteur'] = 'Olja Savičević Ivančević'
df_wikipedia.loc[df_wikipedia['Titre'] == "La Grâce", 'Auteur'] = 'Thibault de Montaigu'
df_wikipedia.loc[df_wikipedia['Titre'] == "Le Voyant d'Étampes", 'Auteur'] = 'Abel Quentin'
df_wikipedia.loc[df_wikipedia['Titre'] == "Héritage", 'Auteur'] = 'Miguel Bonnefoy'
df_wikipedia.loc[(df_wikipedia['Titre'] == 'Les Paupières des poissons'), 'Auteur'] = 'Fanny Vaucher, Sébastien Moro'
df_wikipedia.loc[df_wikipedia['Titre'] == 'Insolente Veggie', 'Auteur'] = 'Rosa B.'
df_wikipedia.loc[df_wikipedia['Titre'] == 'Zita', 'Auteur'] = 'Olivier Hercend'
df_wikipedia.loc[df_wikipedia['Titre'] == "À l’abri des hommes et des choses", 'Auteur'] = 'Stéphanie Boulay'

In [ ]:
df_wikipedia.loc[(df_wikipedia['Prix'] == 'Prix littéraire du Monde') & (df_wikipedia['Année'] == 2020), ['Titre', 'Auteur']] = ['Elle a menti pour les ailes', 'Francesca Serra']
df_wikipedia.loc[(df_wikipedia['Prix'] == 'Prix Eugène-Dabit du roman populiste') & (df_wikipedia['Année'] == 2021), 'Auteur'] =  'Samira Sedira'
df_wikipedia.loc[(df_wikipedia['Titre'] == 'Mon territoire') & (df_wikipedia['Année'] == 2020), ['Prix', 'Auteur']] = ['Grand prix des lectrices (Elle)', 'Tess Sharpe']
df_wikipedia.loc[df_wikipedia['Titre'] == 'Avant que le monde ne se ferme','Auteur'] = 'Alain Mascaro'
df_wikipedia.loc[df_wikipedia['Titre'] == 'Léopold Sédar Senghor','Auteur'] = ' Jean-Pierre Langellier'
df_wikipedia.loc[df_wikipedia['Titre'] == 'La Carte postale','Auteur'] = 'Anne Berest'
df_wikipedia.loc[df_wikipedia['Titre'] == 'Le Passeur','Auteur'] = 'Lois Lowry'
df_wikipedia.loc[df_wikipedia['Titre'] == 'La Muse rouge','Auteur'] = 'Véronique de Haas'
df_wikipedia.loc[df_wikipedia['Titre'] == 'Les Garçons de la cité-jardin','Auteur'] = 'Dan Nisand'
df_wikipedia.loc[df_wikipedia['Titre'] == 'Réacteur 3 ','Auteur'] = 'Ludovic Bernhardt'
df_wikipedia.loc[df_wikipedia['Titre'] == 'Les Enfants endormis','Auteur'] = 'Anthony Passeron'
df_wikipedia.loc[df_wikipedia['Titre'] == 'Georges Perec','Auteur'] = 'Claude Burgelin'
df_wikipedia.loc[df_wikipedia['Titre'] == "Mourir avant que d'apparaître",'Auteur'] = 'Rémi David'

On vérifie qu'il n'y a plus de problème

In [189]:
erreur_recherche = ["Livre pas trouvé", ""]
df_verif =  df_wikipedia[df_wikipedia["Auteur"].isin(erreur_recherche)]
print(df_verif)

Empty DataFrame
Columns: [Année, Prix, Auteur, Titre, prix_19_23]
Index: []


Maintenant, on rajoute une colonne de 1, indiquant que le livre a obtenu un prix.

In [190]:
import pandas as pd

# on ajoute une colonne indicatrice "prix" 
df_wikipedia['prix_19_23'] = 1
print("Colonnes avant sauvegarde :", df.columns)

Colonnes avant sauvegarde : Index(['Année', 'Prix', 'Auteur', 'Titre', 'prix_19_23'], dtype='object')


Désormais, on essaye de simplifier la base afin de n'avoir plus de doublons de livre pour plus de simplicité.

Tout d'abord, nous avons décidé dans la nouvelle base de ne pas garder la colonne prix pour plus de simplicité

In [194]:
df_wikifinal = df_wikipedia.drop(columns=['Prix'])

Maintenant, on s'attelle aux doublons.

In [196]:
df_wikifinal = df_wikifinal.drop_duplicates(subset=['Titre'])

Vérifions que cela a bien marché.

In [197]:
df_wikifinal['Doublons'] = df_wikifinal['Titre'].duplicated(keep=False)  # Repère tous les doublons

# Afficher uniquement les lignes dupliquées
doublons = df_wikifinal[df_wikifinal['Doublons']]
print(doublons)

Empty DataFrame
Columns: [Année, Auteur, Titre, prix_19_23, Doublons]
Index: []


Pour rendre la base propre, on enlève la colonne "Doublons".

In [200]:
df_wikifinal = df_wikifinal.drop(columns=['Doublons'])

Maintenant, on peut modifier le fichier CSV avec les modifications afin de le fusionner plus tard.

In [201]:
df_wikifinal.to_csv("prix_litteraires.csv", index=False)

Partie 3: import des données Youtube. On ne mergera pas la base Youtube avec les autres. Le but sera de compter le nombre d'occurence de chacun livre de la première base dans cette dernière. On a obtenu ces données à partir de l'API disponibles en ligne. Toutefois, la base comportant des données sensibles, nous ne mettrons pas la clé accessible dans le Github, nous la partagerons nous même avec les personnes intéressées par nos données. On se limites aux vidéos sorties entre 2019 et 2023.

In [ ]:
import pandas as pd
import requests

# Remplacez key par sa propre clé API

# Fonction pour récupérer les vidéos avec des dates spécifiques
def get_videos(query, start_date="2019-01-01T00:00:00Z", end_date="2023-12-31T23:59:59Z", max_results=50,key):
    url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&q={query}&type=video&maxResults={max_results}&publishedAfter={start_date}&publishedBefore={end_date}&key={key}'
    response = requests.get(url)  # Requête pour rechercher des vidéos
    data = response.json()

    videos = []
    for video in data['items']:
        video_info = {
            'video_id': video['id']['videoId'],
            'title': video['snippet']['title'],
            'description': video['snippet']['description'],
            'published_at': video['snippet']['publishedAt'],
            'channel_title': video['snippet']['channelTitle']
        }	
        videos.append(video_info)

    # Récupérer les pages suivantes si nécessaire
    next_page_token = data.get('nextPageToken')
    while next_page_token:
        url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&q={query}&type=video&maxResults={max_results}&pageToken={next_page_token}&publishedAfter={start_date}&publishedBefore={end_date}&key={key}'
        response = requests.get(url)
        data = response.json()
        for video in data['items']:
            video_info = {
                'video_id': video['id']['videoId'],
                'title': video['snippet']['title'],
                'description': video['snippet']['description'],
                'published_at': video['snippet']['publishedAt'],
                'channel_title': video['snippet']['channelTitle']
            }
            videos.append(video_info)
        next_page_token = data.get('nextPageToken')

    return videos

# Appel de la fonction pour récupérer les vidéos entre 2019 et 2023
query = "livre"
videos = get_videos(query, start_date="2019-01-01T00:00:00Z", end_date="2023-12-31T23:59:59Z", max_results=50, key)

# Convertir les données en DataFrame
df = pd.DataFrame(videos)

# Sauvegarder le DataFrame sous forme de fichier CSV
df.to_csv('youtube_livre_2019_2023.csv', index=False)

print("Le fichier CSV a été créé avec succès !")

Partie 4: on merge les différentes bases pour simplifier l'analyse

On commence par charger les dataframe à partir des CSV. Cette étape nous permet à l'échelle d'une journée de pouvoir ne pas recharger toutes les bases à chaque fois. Toutefois, nous n'effectuons pas forcément cette ligne de code à chaque fois. Si nous venons de refaire tourner tout le code, on peut skipper cette partie.

In [92]:
import pandas as pd
import numpy as np

In [81]:
df_top_books_fnac_2023 = pd.read_csv("best_sellers_fnac_2023_cleaned.csv")
df_babelio_data = pd.read_csv("23 livres les plus lus Babelio.csv")
df_livraddict_data = pd.read_csv("livraddict_prix_2024.csv")
df_wikifinal = pd.read_csv("prix_litteraires.csv")
df_bibli = pd.read_csv("Les livres les plus empruntés à Paris.csv")

Il est nécessaire de renommer certaines colonnes pour standardiser le tout. Pour le moment, nous gardons les variables type_de_document et Catégorie respectivement des bases de bibliothèque et de Livraddict. Peut-être pourra-t-on s'intéresser très rapidement dans nos statistiques descriptives aux notes des ouvrages selon le genre.

In [82]:
df_bibli = df_bibli.rename(columns={'titre': 'Titre'})
df_bibli_formerge = df_bibli.rename(columns={'reservations': 'Nb réservations bibli'})
df_bibli_formerge = df_bibli_formerge.rename(columns={'auteur': 'Auteur'})
df_bibli_formerge = df_bibli_formerge.drop(columns=['rang', 'Nb réservations bibli', 'Classement bibliothèque'])
df_wikifinal = df_wikifinal.drop(columns=['Année'])

Ensuite, on normalise les titres en mettant tout en majuscule et en enlevant les espaces qui peuvent se créer avant le début ou après la fin de la chaîne de caractères. Le but est qu'il n'y ait pas d'erreur au moment du merge, ou le moins possible.

In [83]:
df_top_books_fnac_2023['Titre'] = df_top_books_fnac_2023['Titre'].str.lower().str.strip()
df_babelio_data['Titre'] = df_babelio_data['Titre'].str.lower().str.strip()
df_livraddict_data['Titre'] = df_livraddict_data['Titre'].str.lower().str.strip()
df_wikifinal['Titre'] = df_wikifinal['Titre'].str.lower().str.strip()
df_bibli_formerge['Titre'] = df_bibli['Titre'].str.lower().str.strip()

Enfin, on fait notre opération merge. On merge les bases mes unes avec les autre l'une après l'autre.
On doit également retravailler à chaque fois la colonne "Auteur". L'idée c'est de créer une colonne unique "Auteur" pour réunir toutes les information d'auteur dans une seule variable on donne un ordre de priorité: Auteur, Auteur_x, Auteur_y
si une colonne contient une valeur manquante (NaN), elle est remplit par la valeur de la colonne suivante.
La méthode .combine_first() remplit les valeurs manquantes (NaN) de la colonne de gauche (merged_df['Auteur']) 
avec les valeurs correspondantes de la colonne de droite (merged_df['Auteur_x']).

In [84]:
merged_df = df_top_books_fnac_2023.merge(df_babelio_data, on="Titre", how="outer")
merged_df['Auteur']= None
merged_df['Auteur'] = merged_df['Auteur'].combine_first(merged_df['Auteur_x']).combine_first(merged_df['Auteur_y'])
merged_df.drop(columns=['Auteur_x', 'Auteur_y'], inplace=True)


In [85]:
merged_df = merged_df.merge(df_livraddict_data, on="Titre", how="outer")
merged_df['Auteur']= None
merged_df['Auteur'] = merged_df['Auteur'].combine_first(merged_df['Auteur_x']).combine_first(merged_df['Auteur_y'])
merged_df.drop(columns=['Auteur_x', 'Auteur_y'], inplace=True)

In [86]:
merged_df = merged_df.merge(df_wikifinal, on="Titre", how="outer")
merged_df['Auteur']= None
merged_df['Auteur'] = merged_df['Auteur'].combine_first(merged_df['Auteur_x']).combine_first(merged_df['Auteur_y'])
merged_df.drop(columns=['Auteur_x', 'Auteur_y'], inplace=True)

In [87]:
merged_df = merged_df.merge(df_bibli_formerge, on="Titre", how="outer")
merged_df['Auteur']= None
merged_df['Auteur'] = merged_df['Auteur'].combine_first(merged_df['Auteur_x']).combine_first(merged_df['Auteur_y'])
merged_df.drop(columns=['Auteur_x', 'Auteur_y'], inplace=True)

In [88]:
merged_df.head(15)

,Titre,top_fnac_1,top_fnac_2_plus,top_babelio,Catégorie,top_livraddict,prix_19_23,type_de_document,Top bibliothèque,Auteur
0,(très) cher cinéma français,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Éric Neuhoff
1,555,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Hélène Gestern
2,adieu birkenau,1.0,0.0,NaN,Bd,1.0,NaN,NaN,NaN,Ginette Kolinka
3,adieu tanger,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Salma El Moumni
4,adios cow boy,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Olja Savičević Ivančević
5,anatèm,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Neal Stephenson
6,apeirogon,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Colum McCann
7,arcadie,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Emmanuelle Bayamack-Tam
8,argonne,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Stéphane Emond
9,astérix,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Hors collection : L’Empire du milieu – Fabrice...


Désormais, on remplace toutes les valeurs None par des 0 afin d'avoir une base propre

In [93]:
def place0(row):
    global column
    if np.isnan(row[column]):
        value = 0
    else:
        value = row[column]
    return value

In [94]:
columns = ['top_fnac_1', 'top_fnac_2_plus', 'top_babelio', 'top_livraddict', 'prix_19_23', 'Top bibliothèque']
for i in range(0,len(columns)):
    column = columns[i]
    merged_df[column] = merged_df.apply(place0, axis=1)

In [95]:
merged_df.head(8)

,Titre,top_fnac_1,top_fnac_2_plus,top_babelio,Catégorie,top_livraddict,prix_19_23,type_de_document,Top bibliothèque,Auteur
0,(très) cher cinéma français,0.0,0.0,0.0,NaN,0.0,1.0,NaN,0.0,Éric Neuhoff
1,555,0.0,0.0,0.0,NaN,0.0,1.0,NaN,0.0,Hélène Gestern
2,adieu birkenau,1.0,0.0,0.0,Bd,1.0,0.0,NaN,0.0,Ginette Kolinka
3,adieu tanger,0.0,0.0,0.0,NaN,0.0,1.0,NaN,0.0,Salma El Moumni
4,adios cow boy,0.0,0.0,0.0,NaN,0.0,1.0,NaN,0.0,Olja Savičević Ivančević
5,anatèm,0.0,0.0,0.0,NaN,0.0,1.0,NaN,0.0,Neal Stephenson
6,apeirogon,0.0,0.0,0.0,NaN,0.0,1.0,NaN,0.0,Colum McCann
7,arcadie,0.0,0.0,0.0,NaN,0.0,1.0,NaN,0.0,Emmanuelle Bayamack-Tam
